# NHANES data exploration

https://www.cdc.gov/nchs/nhanes/about/index.html

In [1]:
import pandas as pd

## Body Measures

In [3]:

url = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2021/DataFiles/BMX_L.xpt"
pd.read_sas(url)

,SEQN,BMDSTATS,BMXWT,BMIWT,BMXRECUM,BMIRECUM,BMXHEAD,BMIHEAD,BMXHT,BMIHT,...,BMXLEG,BMILEG,BMXARML,BMIARML,BMXARMC,BMIARMC,BMXWAIST,BMIWAIST,BMXHIP,BMIHIP
0,130378.0,1.0,86.9,NaN,NaN,NaN,NaN,NaN,179.5,NaN,...,42.8,NaN,42.0,NaN,35.7,NaN,98.3,NaN,102.9,NaN
1,130379.0,1.0,101.8,NaN,NaN,NaN,NaN,NaN,174.2,NaN,...,38.5,NaN,38.7,NaN,33.7,NaN,114.7,NaN,112.4,NaN
2,130380.0,1.0,69.4,NaN,NaN,NaN,NaN,NaN,152.9,NaN,...,38.5,NaN,35.5,NaN,36.3,NaN,93.5,NaN,98.0,NaN
3,130381.0,1.0,34.3,NaN,NaN,NaN,NaN,NaN,120.1,NaN,...,NaN,NaN,25.4,NaN,23.4,NaN,70.4,NaN,NaN,NaN
4,130382.0,3.0,13.6,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,1.0,NaN,1.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8855,142306.0,1.0,25.3,NaN,NaN,NaN,NaN,NaN,128.0,NaN,...,32.0,NaN,25.0,NaN,19.0,NaN,57.7,NaN,NaN,NaN
8856,142307.0,3.0,NaN,1.0,NaN,NaN,NaN,NaN,143.8,NaN,...,NaN,1.0,34.0,NaN,35.4,NaN,NaN,1.0,NaN,1.0
8857,142308.0,1.0,79.3,NaN,NaN,NaN,NaN,NaN,173.3,NaN,...,41.8,NaN,40.0,NaN,30.6,NaN,98.4,NaN,97.7,NaN
8858,142309.0,1.0,81.9,NaN,NaN,NaN,NaN,NaN,179.1,NaN,...,44.0,NaN,40.0,NaN,30.8,NaN,96.0,NaN,103.3,NaN


## Files required for calculation of phenotypic age
- publication with formula for phynotypic age: https://pmc.ncbi.nlm.nih.gov/articles/PMC5940111/

In [20]:
# the newest data dont have all required data such as Albuminin, use older data

base_url = "https://wwwn.cdc.gov/Nchs/Data/Nhanes/Public/2017/DataFiles/"

name2file = {
    "Albumin": "BIOPRO_J.xpt",  # g/L
    "Creatinine": "BIOPRO_J.xpt",  # umol/L
    "Glucose_serum": "BIOPRO_J.xpt",  # mmol/L
    "C_reactive_protein_log": "HSCRP_J.xpt",  # mg/dL
    "Lymphocyte_percent": "CBC_J.xpt",  # %
    "Mean_cell_volume": "CBC_J.xpt",  # fL
    "Red_cell_distribution_width": "CBC_J.xpt",  # %
    "Alkaline_phosphatase": "BIOPRO_J.xpt",  # U/L
    "White_blood_cell_count": "CBC_J.xpt",  # 1000 cells/uL
    "Age_years": "DEMO_J.xpt",  # Years
}

name2entry = {
    "Albumin": "LBDSALSI",  # g/L
    "Creatinine": "LBDSCRSI",  # umol/L
    "Glucose_serum": "LBDSGLSI",  # mmol/L
    "C_reactive_protein_log": "LBXHSCRP",  # TODO: mg/L -> log(mg/dL)
    "Lymphocyte_percent": "LBXLYPCT",  # %
    "Mean_cell_volume": "LBXMCVSI",  # fL
    "Red_cell_distribution_width": "LBXRDW",  # %
    "Alkaline_phosphatase": "LBXSAPSI",  # IU/L == U/L
    "White_blood_cell_count": "LBXWBCSI",  # 1000 cells/uL
    "Age_years": "RIDAGEYR",  # Years
}


url = base_url+name2file["C_reactive_protein_log"]
pd.read_sas(url)

,SEQN,LBXHSCRP,LBDHRPLC
0,93703.0,NaN,NaN
1,93704.0,0.29,5.397605e-79
2,93705.0,2.72,5.397605e-79
3,93706.0,0.74,5.397605e-79
4,93707.0,0.32,5.397605e-79
...,...,...,...
8361,102952.0,0.45,5.397605e-79
8362,102953.0,1.84,5.397605e-79
8363,102954.0,0.81,5.397605e-79
8364,102955.0,5.11,5.397605e-79


## Read all files, merge and filter for relevant variables

In [21]:
import numpy as np
unique_files = np.unique(list(name2file.values()))
print(unique_files)

['BIOPRO_J.xpt' 'CBC_J.xpt' 'DEMO_J.xpt' 'HSCRP_J.xpt']


In [ ]:
pd.read_sas(base_url + unique_files[3])  # manually test downloading all files

,SEQN,LBXHSCRP,LBDHRPLC
0,93703.0,NaN,NaN
1,93704.0,0.29,5.397605e-79
2,93705.0,2.72,5.397605e-79
3,93706.0,0.74,5.397605e-79
4,93707.0,0.32,5.397605e-79
...,...,...,...
8361,102952.0,0.45,5.397605e-79
8362,102953.0,1.84,5.397605e-79
8363,102954.0,0.81,5.397605e-79
8364,102955.0,5.11,5.397605e-79


In [29]:
# download all files to data directory
import requests
import os

def download_file(url, filename):
    response = requests.get(url)
    with open(filename, mode="wb") as file:
        file.write(response.content)

data_path = os.path.join("..", "data")
for file in unique_files:
    os.makedirs(data_path, exist_ok=True)
    download_file(base_url+file, os.path.join(data_path, file))

In [37]:
df = pd.read_sas(os.path.join(data_path, unique_files[3]))
counts = df["SEQN"].value_counts()
print(f"counts > 1 ={counts[counts > 1]}")

counts > 1 =Series([], Name: count, dtype: int64)


=> all SEQN entries seems to be unique => unique values

In [39]:
# try to merge based on SEQN/patient
raw_dfs = [ pd.read_sas(os.path.join(data_path, file)) for file in unique_files ]
result_df = raw_dfs[0]
for raw_df in raw_dfs[1:0]:
    result_df = result_df.merge(raw_df, on="SEQN")

In [40]:
result_df

,SEQN,LBXSATSI,LBDSATLC,LBXSAL,LBDSALSI,LBXSAPSI,LBXSASSI,LBXSC3SI,LBXSBU,LBDSBUSI,...,LBXSCA,LBDSCASI,LBXSCH,LBDSCHSI,LBXSTP,LBDSTPSI,LBXSTR,LBDSTRSI,LBXSUA,LBDSUASI
0,93705.0,16.0,5.397605e-79,4.4,44.0,74.0,20.0,31.0,11.0,3.93,...,9.2,2.300,157.0,4.060,7.3,73.0,95.0,1.073,5.8,345.0
1,93706.0,10.0,5.397605e-79,4.4,44.0,79.0,14.0,28.0,12.0,4.28,...,9.6,2.400,149.0,3.853,7.1,71.0,92.0,1.039,8.0,475.8
2,93707.0,13.0,5.397605e-79,5.2,52.0,238.0,24.0,22.0,17.0,6.07,...,10.1,2.525,199.0,5.146,8.0,80.0,110.0,1.242,5.5,327.1
3,93708.0,19.0,5.397605e-79,3.9,39.0,66.0,21.0,27.0,16.0,5.71,...,9.5,2.375,210.0,5.431,7.1,71.0,72.0,0.813,4.5,267.7
4,93709.0,15.0,5.397605e-79,3.7,37.0,86.0,17.0,24.0,20.0,7.14,...,9.9,2.475,180.0,4.655,7.0,70.0,132.0,1.490,6.2,368.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6396,102952.0,22.0,5.397605e-79,4.8,48.0,57.0,27.0,27.0,13.0,4.64,...,9.9,2.475,121.0,3.129,7.4,74.0,98.0,1.106,6.4,380.7
6397,102953.0,40.0,5.397605e-79,4.0,40.0,115.0,29.0,24.0,17.0,6.07,...,9.0,2.250,184.0,4.758,7.5,75.0,114.0,1.287,5.8,345.0
6398,102954.0,6.0,5.397605e-79,3.9,39.0,55.0,15.0,21.0,8.0,2.86,...,9.0,2.250,172.0,4.448,6.5,65.0,64.0,0.723,3.1,184.4
6399,102955.0,11.0,5.397605e-79,4.1,41.0,84.0,16.0,23.0,11.0,3.93,...,9.2,2.300,151.0,3.905,7.5,75.0,89.0,1.005,6.3,374.7
